In [126]:
from Module.Record import record
from Module.train import finetune
from Module.Results import correctly_identify, could_not_identify
from Module.env import inference

In [127]:
#Imports
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from sklearn.model_selection import train_test_split

from pyannote.audio import Inference
# from scipy.spatial.distance import cdist
# instantiate pretrained model
from pyannote.audio import Model

import pyaudio
import wave
import json

In [128]:
#dict for all the recorded names
names={}

#data folder path
folder_path = "E:/New-Codes/Repositories/IdentifyMe-AI/VoiceData/data" 

In [129]:

# Define parameters
FORMAT = pyaudio.paInt16   # Format of audio
CHANNELS = 1               # Number of channels (1 for mono, 2 for stereo)
RATE = 44100               # Sample rate
CHUNK = 1024               # Buffer size
RECORD_SECONDS = 5        # Duration of recording



# To record the audio
def record(audio):
    # Create an interface to PortAudio
    

    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    print("Recording...")

    frames = []

    for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Finished recording.")

    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Save the recorded data as a WAV file
    with wave.open("recorded.wav" , 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print("Recorded audio saved as recorded.wav ")
    return frames



In [130]:
def finetune(X_new,y_new):

    X_old=np.load("X_embeddings.npy")
    y_old=np.load("y_labels.npy")


    X_finetune = np.concatenate([X_old, X_new.reshape(1,-1)], axis=0)
    y_finetune = np.concatenate([y_old, y_new.reshape(1)], axis=0)

    model=load_model("updated_model.keras")

    history = model.fit(X_finetune, y_finetune, epochs=10, batch_size=8, validation_split=0.2)

    model.save("updated_model.keras")



In [131]:
#if couldnt identify
def could_not_identify(embeddings,audio,frames,names):
    

    que=input(f"Are you a new Speaker?   (y/n)")
    is_new= False if (que=="n" or que=="n") else True
    new_name="Prashasst"
    new_label=0

    if not is_new:
        print(names)
        new_label=int(input("Enter your id : "))
        new_name=names[new_label]
        np.save("X_new_embeddings.npy",embeddings)
        np.save("y_new_labels.npy",new_label)
        finetune(embeddings,np.array(new_label))
    else:
        new_name=input("Enter your correct name: ")
        new_label= list(names.keys())[-1]+1
        new_name_folder_path=f"{folder_path}/{new_label}_{new_name}"
        os.makedirs(new_name_folder_path)

        print(" Retrain the model from scratch ")
    

    # new_name_folder_path=f"{folder_path}/{new_label}_{new_name}"

    # if new_name in names:
    #     print("It looks like your name is already saved.")
    #     new_label=names[new_name]

    

    new_name_folder_path=f"{folder_path}/{new_label}_{new_name}"

    # Saving the recorded data as a WAV file
    with wave.open(f"{new_name_folder_path}/{np.random.randint(100)}_{new_name}.wav" , 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print(f"Recorded audio saved in {new_label}_{new_name} in voiceData/data folder")


In [132]:
#if identifued correctly
def correctly_identify(embeddings,the_label,audio,frames,names):

    
    
    que=input(f"Were you identifed correctly? (y/n)")
    correct= True if (que=="y" or que=="Y") else False

    if not correct:
        print(names)
        the_label=int(input("Enter your correct id  or -1 if not here: "))
        if the_label==-1:return
        the_name=names[the_label]


    the_name=names[the_label]

    np.save("X_new_embeddings.npy",embeddings)
    np.save("y_new_labels.npy",the_label)

    finetune(embeddings,np.array(the_label))

    

    the_name_folder_path=f"{folder_path}/{the_label}_{the_name}"

   

    # Saving the recorded data as a WAV file
    with wave.open(f"{the_name_folder_path}/{np.random.randint(100)}_{the_name}.wav" , 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print(f"Recorded audio saved in {the_label}_{the_name} in voiceData/data folder")


In [133]:
# new_label= list(names.keys())[-1]+1
# print(new_label)

In [134]:
# for extracting vector embeddings from the audio files 

model = Model.from_pretrained("pyannote/wespeaker-voxceleb-resnet34-LM",cache_dir=r"C:\Users\prash\.cache\huggingface\hub\models--pyannote--wespeaker-voxceleb-resnet34-LM")
inference = Inference(model, window="whole")

In [135]:
#making the X and y for model


final=[]
y_label=[]
for labeled_name in os.listdir(folder_path):
    label=int(labeled_name[0])
    name =labeled_name[2:]

    if label not in names:names[label]=name
    
    
    file_path=f"{folder_path}/{labeled_name}/"
    # print(file_path)
    for wav_file in os.listdir(file_path):
        # print(f"{file_path}{wav_file}")
        final.append(inference(f"{file_path}{wav_file}"))
        y_label.append(label)

X=np.array(final)
y=np.array(y_label)

# Saving the labels to a json
with open('names_identified.json', 'w') as f:
    json.dump(names, f)


In [136]:
names

{0: 'Prashasst',
 1: 'Sarthak',
 2: 'Satendra',
 3: 'Shraddha',
 4: 'Shreyansh',
 5: 'Sourabh',
 6: 'Shivam_Sharma',
 7: 'Rupesh',
 8: 'Shivam_Devrani'}

In [137]:
print(f"X shape: {X.shape} y shape: {y.shape}")

X shape: (75, 256) y shape: (75,)


In [138]:
np.save("X_embeddings.npy",X)
np.save("y_labels.npy",y)

In [139]:
#spliting into training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=49)

In [156]:


# Define the model
model = tf.keras.Sequential([
    InputLayer(shape=(256,), name='input_layer'),  
    tf.keras.layers.Reshape((1, 256, 1)),  

    # Convolutional layers
    Conv2D(32, (1, 3), activation='relu', padding='same'),  # 1x3 convolution for embedding
    MaxPooling2D(pool_size=(1, 2)),  # Downsample

    Conv2D(64, (1, 3), activation='relu', padding='same'),  # 1x3 convolution
    MaxPooling2D(pool_size=(1, 2)),  # Downsample

    Conv2D(128, (1, 3), activation='relu', padding='same'),  # 1x3 convolution
    MaxPooling2D(pool_size=(1, 2)),  # Downsample

    Flatten(),  # Flatten the output for dense layers

    # Fully connected layers
    Dense(256, activation='relu'),
    Dropout(0.5),  # Regularization to prevent overfitting
    Dense(128, activation='relu'),
    Dropout(0.5),  # Regularization
    Dense(len(names), activation='softmax')  # Output layer for classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_4 (Reshape)             │ (None, 1, 256, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 1, 256, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 1, 128, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 1, 128, 64)     │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 1, 64, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 1, 64, 128)     │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 1, 32, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │     1,048,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,113,929 (4.25 MB)

 Trainable params: 1,113,929 (4.25 MB)

 Non-trainable params: 0 (0.00 B)

In [157]:
history = model.fit(X_train, y_train,
                    epochs=100,            # Adjust the number of epochs as needed
                    batch_size=32,        # Adjust based on your data size
                    validation_data=(X_val, y_val))

Epoch 1/100


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 916ms/step - accuracy: 0.1790 - loss: 2.1531 - val_accuracy: 0.2500 - val_loss: 2.2266
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.2973 - loss: 2.0467 - val_accuracy: 0.2500 - val_loss: 2.5574
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.3282 - loss: 2.0937 - val_accuracy: 0.2500 - val_loss: 2.5611
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2973 - loss: 2.0818 - val_accuracy: 0.2500 - val_loss: 2.4040
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3400 - loss: 1.9563 - val_accuracy: 0.2500 - val_loss: 2.3410
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.2401 - loss: 2.0486 - val_accuracy: 0.2500 - val_loss: 2.2939
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2671 - loss: 1.9829 - val_accuracy: 0.2500 - val_loss: 2.2309
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.3549 - loss: 1.9229 - val_accuracy: 0.2500 - val_loss: 2.2216
Epo

In [158]:
model.save("updated_model.keras")

In [143]:
import json
from tensorflow.keras.models import load_model

with open("names_identified.json", 'r') as f:
    loaded_json = json.load(f)
identified_names={int(k):v for k,v in loaded_json.items()}

model=load_model("updated_model.keras")

In [144]:

from sklearn.metrics import accuracy_score

def build_test(test_path, names):
    X_temp=[]
    y_temp=[]

    for wav_file in os.listdir(test_path):
        label=int(wav_file[0])
        name =wav_file[2:]

        if label not in names:
            print(f"{name} is not in our database")
        
        
        X_temp.append(inference(f"{test_path}\\{wav_file}"))
        y_temp.append(label)

    X_test=np.array(X_temp)
    y_test=np.array(y_temp)
    print(f"X_test shape: {X_test.shape} y_test shape: {y_test.shape}")
    return X_test,y_test

def the_accuracy(model,X_test,y_test):
    y_pred = model.predict(X_test)  # This will give probabilities for each class

    # Convert predictions to class labels (using argmax for multiclass classification)
    y_pred_labels = y_pred.argmax(axis=1)
    # print(y_pred_labels)

    
    accuracy = accuracy_score(y_test, y_pred_labels)

    
    return accuracy



In [155]:
test_path=r"E:\New-Codes\Repositories\IdentifyMe-AI\VoiceData\Test"

X_test,y_test = build_test(test_path=test_path, names=identified_names)

accuracy = the_accuracy(model=model,X_test=X_test,y_test=y_test)

print(f"Model Accuracy: {accuracy * 100:.2f}%")

X_test shape: (6, 256) y_test shape: (6,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
Model Accuracy: 100.00%


In [146]:
#testing the model




audio = pyaudio.PyAudio()
frames=record(audio=audio)

test=inference(r"E:\New-Codes\Repositories\IdentifyMe-AI\recorded.wav")
# test=inference(r"E:\New-Codes\Repositories\IdentifyMe-AI\VoiceData\Test\0_Prashasst_T1_01.wav")

predection=model.predict(test.reshape(1,-1))


threshold = 0.7 
certainty= np.floor(np.max(predection)*100)

# Checking if the highest probability exceeds the threshold
result = np.argmax(predection) if  np.max(predection) > threshold else -1
uncertain= np.argmax(predection)


identified= True if result!=-1 else False


Recording...
Finished recording.
Recorded audio saved as recorded.wav 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step


In [147]:
if result==-1:print(f"Could not identify You its only {certainty}% certain but you sound like {identified_names[uncertain]}")
else:print(f"Person identified as {identified_names[result]} with {certainty} % Certainty")

Person identified as Prashasst with 99.0 % Certainty


In [148]:
# print( f"old {X.shape} new {test.reshape(1,-1).shape}")
# print( f"old {y.shape} new {result.reshape(1).shape}")

In [149]:
#if could not identify

if not identified: could_not_identify(embeddings=test,audio=audio,frames=frames,names=identified_names)
else:correctly_identify(embeddings=test,the_label=result,audio=audio,frames=frames,names=identified_names)

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - accuracy: 1.0000 - loss: 0.0036 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9801 - loss: 0.0707 - val_accuracy: 1.0000 - val_loss: 6.2890e-04
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9963 - loss: 0.0105 - val_accuracy: 1.0000 - val_loss: 9.6314e-04
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9495 - loss: 0.2039 - val_accuracy: 1.0000 - val_loss: 0.0041
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9812 - loss: 0.0531 - val_accuracy: 1.0000 - val_loss: 0.0115
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 0.0052 - val_accuracy: 1.0000 - val_loss: 0.0204
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.0280 - val_accuracy: 1.0000 - val_loss: 0.0149
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9943 - loss: 0.0284 - val_accuracy: 1.0000 - val_loss: